# czce

different link after 20151006

In [26]:
from grab import Grab
from datetime import timedelta, date
import time
from random import randint


def date_range(start_date, end_date):
    """
    find business days to parse
    :param start_date:
    :param end_date:
    :return: generator with business days
    """
    for n in range(int((end_date - start_date).days) + 1):
        if (start_date + timedelta(n)).weekday() not in (5, 6):
            yield start_date + timedelta(n)


def find_links(dates):
    """
    create links
    :param dates: days range when we need to parse data
    :return: list with links
    """
    _links = []
    # old link format
    # url = 'http://english.czce.com.cn/enportal/exchange/marketdata/{0}/datadaily/{1}.htm'
    # new link format from 20151008
    url = 'http://english.czce.com.cn/enportal/DFSStaticFiles/Future/{0}/{1}/EnglishFutureDataDaily.htm'
    for single_date in dates:
        _links.append(url.format(single_date.strftime("%Y"), single_date.strftime("%Y%m%d")))
    return _links


def parse_data(linkz):
    """
    parse data from each link
    :param linkz: list with links
    :return: list with data
    """
    g = Grab()
    _data = []
    for l in linkz:
        try:
            g.go(l)
            for i in g.doc.select('//*[@id="senfe"]/tr'):
                if 'Total' in i.text():
                    # old format
                    # _data.append((l[-12:-4], i.text().split(' ')[1], i.text().split(' ')[2]))
                    # print('added', l[-12:-4])
                    # new format
                    _data.append((l[-35:-27], i.text().split(' ')[1], i.text().split(' ')[2]))
                    print('added', l[-35:-27])
            time.sleep(randint(0, 3))  # try to avoid ban
        except:
            print('error', l[-12:-4])
            continue
    return _data


def save_to_file(data_save):
    with open("czce_new.txt", "w") as f:
        f.write("(Date, Total_Volume, Total_Open_Interest)")
        f.write('\n')
        for raw in data_save:
            f.write(str(raw))
            f.write('\n')
    print("Saved to file {}".format(f.name))


if __name__ == '__main__':
    start = date(2015, 10, 8)
    end = date(2016, 8, 21)
    links = find_links(date_range(start, end))
    data = parse_data(links)
    save_to_file(data)

added 20100113
added 20100114
added 20100115
added 20100118
added 20100119
added 20100120
added 20100121
added 20100122
added 20100125
added 20100126
added 20100127
added 20100128
added 20100129
added 20100201
added 20100202
added 20100203
added 20100204
added 20100205
added 20100208
added 20100209
added 20100210
added 20100211
added 20100212
added 20100215
added 20100216
added 20100217
added 20100218
added 20100219
added 20100222
added 20100223
added 20100224
added 20100225
added 20100226
added 20100301
added 20100302
added 20100303
added 20100304
added 20100305
added 20100308
added 20100309
added 20100310
added 20100311
added 20100312
added 20100315
added 20100316
added 20100317
added 20100318
added 20100319
added 20100322
added 20100323
added 20100324
added 20100325
added 20100326
added 20100329
added 20100330
added 20100331
added 20100401
added 20100402
added 20100405
added 20100406
added 20100407
added 20100408
added 20100409
added 20100412
added 20100413
added 20100414
added 2010

# shfe

In [ ]:
from datetime import timedelta, date
import time
from random import randint
import requests


def date_range(start_date, end_date):
    """
    find business days to parse
    :param start_date:
    :param end_date:
    :return: generator with business days
    """
    for n in range(int((end_date - start_date).days) + 1):
        if (start_date + timedelta(n)).weekday() not in (5, 6):
            yield start_date + timedelta(n)


def find_links(dates):
    """
    create links
    :param dates: days range when we need to parse data
    :return: list with links
    """
    _links = []
    url = 'http://www.shfe.com.cn/data/dailydata/kx/kx{}.dat'
    for single_date in dates:
        _links.append(url.format(single_date.strftime("%Y%m%d")))
    return _links


def parse_data(linkz):
    """
    parse data from each link
    :param linkz: list with links
    :return: list with data
    """
    _data = []
    for i, l in enumerate(linkz):
        try:
            r = requests.get(l).json()
            time.sleep(randint(0, 3))  # try to avoid ban
            _data.append((l[-12:-4], r['o_curinstrument'][-1]['VOLUME'],
                         r['o_curinstrument'][-1]['OPENINTEREST']))
            print('added', l[-12:-4])
            if i % 3 == 0:
                save_to_file(_data)
                _data.clear()
        except:
            print('error', l[-12:-4])
            continue
    save_to_file(_data)
    return _data


def save_to_file(data_save):
    with open("shfe.txt", "a") as f:
        #f.write("(Date, Total_Volume, Total_Open_Interest)")
        #f.write('\n')
        for raw in data_save:
            f.write(str(raw))
            f.write('\n')
    print("Saved to file {}".format(f.name))


if __name__ == '__main__':
    start = date(2016, 4, 11)
    end = date(2016, 8, 22)
    links = find_links(date_range(start, end))
    data = parse_data(links)
    #save_to_file(data)

# DCE

In [ ]:
from grab import Grab
from datetime import timedelta, date
import time
from random import randint


def date_range(start_date, end_date):
    """
    find business days to parse
    :param start_date:
    :param end_date:
    :return: generator with business days
    """
    for n in range(int((end_date - start_date).days) + 1):
        if (start_date + timedelta(n)).weekday() not in (5, 6):
            yield start_date + timedelta(n)


def grab_data(dates):
    g = Grab()
    url = 'http://www.dce.com.cn/PublicWeb/MainServlet'

    res = []
    for indx, date in enumerate(dates):
        try:
            # post request
            cur_date = int(date.strftime("%Y%m%d"))
            _data = {'action': 'Pu00231_result',
                'Pu00231_Input.trade_date': cur_date,
                'Pu00231_Input.variety': 'all',
                'Pu00231_Input.trade_type': 0,
                'Submit': 'Go'}
            g.go(url, post=_data)
            time.sleep(randint(0, 3))  # try to avoid ban
            # find 'total' element and grab its Volume and OI
            for i in g.doc.select('//body/table/tr/td/*/tr'):
                if 'Total' in i.text():
                    d = i.text().split(' ')
                    res.append((cur_date, d[1], d[2]))
            print('added', cur_date)
            # save all, every 3rd element
            if indx % 3 == 0:
                save_to_file(res)
                res.clear()
        except:
            print('error', cur_date)
            continue
    save_to_file(res)

    
def save_to_file(data_save):
    with open("dce.txt", "a") as f:
        #f.write("(Date, Total_Volume, Total_Open_Interest)")
        #f.write('\n')
        for raw in data_save:
            f.write(str(raw))
            f.write('\n')
    print("Saved to file {}".format(f.name))


if __name__ == '__main__':
    start = date(2016, 8, 15)
    end = date(2016, 8, 21)
    grab_data(date_range(start, end))

# cffex

In [ ]:
from grab import Grab
from datetime import timedelta, date


def date_range(start_date, end_date):
    """
    find business days to parse
    :param start_date:
    :param end_date:
    :return: generator with business days
    """
    for n in range(int((end_date - start_date).days) + 1):
        if (start_date + timedelta(n)).weekday() not in (5, 6):
            yield start_date + timedelta(n)


def save_to_file(data_save):
    with open("cffex.txt", "a") as f:
        # f.write("(Date, Total_Volume, Total_Open_Interest)")
        # f.write('\n')
        for raw in data_save:
            f.write(str(raw))
            f.write('\n')
    print("Saved to file {}".format(f.name))


def create_links(dates):
    url = 'http://www.cffex.com.cn/fzjy/mrhq/{0}/{1}/index.xml'
    _links = []

    for date in dates:
        _links.append(url.format(date.strftime("%Y%m"), date.strftime("%d")))
    return _links


def grab_data(links):
    g = Grab()
    _data = []

    for indx, l in enumerate(links):
        try:
            g.go(l)
            tree = g.doc.build_xml_tree()

            daily_vol = []
            daily_oi = []

            for i in tree:
                for j in i.iter():
                    if j.tag == 'volume':
                        # print(j.tag, ' = ', j.text)
                        vol = j.text
                        daily_vol.append(int(vol.split('.')[0]))
                    elif j.tag == 'openinterest':
                        # print(j.tag, ' = ', j.text)
                        oi = j.text
                        daily_oi.append(int(oi.split('.')[0]))
                        # print(sum(daily_vol), sum(daily_oi))
            _data.append((l[-19:-13] + l[-12:-10], sum(daily_vol), sum(daily_oi)))
            daily_vol.clear()
            daily_oi.clear()
            print('added: ', _data[-1])
            if indx % 3 == 0:
                save_to_file(_data)
                _data.clear()
        except:
            print('error ', l[-19:-13] + l[-12:-10])
            continue
    save_to_file(_data)


if __name__ == '__main__':
    start = date(2014, 8, 17)
    end = date(2016, 8, 22)
    all_links = create_links(date_range(start, end))
    grab_data(all_links)

# szse main_board

In [ ]:
from grab import Grab
from datetime import timedelta, date
import time
from random import randint


def date_range(start_date, end_date):
    """
    find business days to parse
    :param start_date:
    :param end_date:
    :return: generator with business days
    """
    for n in range(int((end_date - start_date).days) + 1):
        if (start_date + timedelta(n)).weekday() not in (5, 6):
            yield start_date + timedelta(n)


def grab_data(dates):
    g = Grab()
    url = 'http://www.szse.cn/szseWeb/FrontController.szse?randnum=0.06895916919770584'

    res = []
    for indx, date in enumerate(dates):
        try:
            # post request
            cur_date = date.strftime("%Y-%m-%d")
            post_data = {'ACTIONID':7,
                        'AJAX':'AJAX-TRUE',
                        'CATALOGID':1849,
                        'txtQueryDate':cur_date,
                        'TABKEY':'tab2'}
            g.go(url, post=post_data)
            time.sleep(randint(0, 3))  # try to avoid ban
            # find 'shares traded' element and grab its Volume
            volume = []
            for tr in g.doc.select('//table[@class="cls-data-table"]/tr'):
                if 'Shares' in tr.text():
                    for td in tr.select('td'):
                        volume.append(td.text())
            print(cur_date,volume[1])
            res.append((cur_date, volume[1]))
            # save all, every 3rd element
            if indx % 3 == 0:
                save_to_file(res)
                res.clear()
        except:
            print('error', cur_date)
            continue
    save_to_file(res)

    
def save_to_file(data_save):
    with open("szse_main.txt", "a") as f:
        for raw in data_save:
            f.write(str(raw))
            f.write('\n')
    print("Saved to file {}".format(f.name))


if __name__ == '__main__':
    start = date(2014, 8, 15)
    end = date(2016, 8, 24)
    grab_data(date_range(start, end))

# szse SME board

In [ ]:
from grab import Grab
from datetime import timedelta, date
import time
from random import randint


def date_range(start_date, end_date):
    """
    find business days to parse
    :param start_date:
    :param end_date:
    :return: generator with business days
    """
    for n in range(int((end_date - start_date).days) + 1):
        if (start_date + timedelta(n)).weekday() not in (5, 6):
            yield start_date + timedelta(n)


def grab_data(dates):
    g = Grab()
    url = 'http://www.szse.cn/szseWeb/FrontController.szse?randnum=0.06895916919770584'

    res = []
    for indx, date in enumerate(dates):
        try:
            # post request
            cur_date = date.strftime("%Y-%m-%d")
            post_data = {'ACTIONID':7,
                        'AJAX':'AJAX-TRUE',
                        'CATALOGID':1849,
                        'txtQueryDate':cur_date,
                        'TABKEY':'tab3'}
            g.go(url, post=post_data)
            time.sleep(randint(0, 3))  # try to avoid ban
            # find 'shares traded' element and grab its Volume
            volume = []
            for tr in g.doc.select('//table[@class="cls-data-table"]/tr'):
                if 'Shares' in tr.text():
                    for td in tr.select('td'):
                        volume.append(td.text())
            print(cur_date,volume[1])
            res.append((cur_date, volume[1]))
            # save all, every 3rd element
            if indx % 3 == 0:
                save_to_file(res)
                res.clear()
        except:
            print('error', cur_date)
            continue
    save_to_file(res)

    
def save_to_file(data_save):
    with open("szse_sme.txt", "a") as f:
        for raw in data_save:
            f.write(str(raw))
            f.write('\n')
    print("Saved to file {}".format(f.name))


if __name__ == '__main__':
    start = date(2014, 8, 15)
    end = date(2016, 8, 24)
    grab_data(date_range(start, end))